<a href="https://colab.research.google.com/github/mzrd91/Machine-Learning-Approaches-for-Static-Modeling-of-SOFC-Using-GP-and-ANN/blob/main/ML_for_GA_SOFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.7 MB/s eta 0:00:00


In [2]:
import math
import random
import operator
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from deap import algorithms, base, creator, tools, gp
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Parameters used in the SOFC stack
number_of_stacks = 1
temperature_values = [873, 973, 1073, 1173, 1273] # List of temperature values in Kelvin
E0 = 1.18 # V
N0 = 384
u = 0.8
Kr = 0.993e-3 # mol/(s A)
Kr_cell = Kr / N0
KH2 = 0.843 # mol/(s atm)
KH2O = 0.281 # mol/(s atm)
KO2 = 2.52 # mol/(s atm)
r = 0.126 # U
rcell = r / N0
rHO = 1.145
i0_den = 20 # mA/cm2
ilimit_den = 900 # mA/cm2
A = 1000 # cm2
n = 2

F = 96485

alpha = 0.5
R = 0.0821 # atm/(mol K)
N = 20

PH2 = 1.265
PO2 = 2.527
PH2O = 0.467

In [ ]:
def calculate_statistics(data):
    best = np.min(data)
    worst = np.max(data)
    mean = np.mean(data)
    variance = np.var(data)
    return best, worst, mean, variance

def evaluate(individual, x_train, y_train, x_test, y_test):
    func = gp.compile(expr=individual, pset=pset)
    y_pred_train = [func(T, current) for T, current in x_train]
    y_pred_test = [func(T, current) for T, current in x_test]
    mae_train = np.mean(np.abs(np.array(y_pred_train) - y_train))
    mae_test = np.mean(np.abs(np.array(y_pred_test) - y_test))
    mse_train = np.mean((np.array(y_pred_train) - y_train) ** 2)
    mse_test = np.mean((np.array(y_pred_test) - y_test) ** 2)
    return mae_train, mse_train, mae_test, mse_test

def generate_data(num_samples=1000):
    current_range = range(20, 900)
    train_currents = random.sample(current_range, int(0.8 * len(current_range))) # 80% for training
    test_currents = random.sample(current_range, int(0.2 * len(current_range))) # 20% for testing
    X_train, Y_train, X_test, Y_test = [], [], [], []

    for T in temperature_values:
        for current in train_currents:
            # Introduce randomness by perturbing temperature and current
            perturbed_T = T + random.uniform(-1, 1) # Add random noise to temperature
            perturbed_current = current + random.uniform(-10, 10) # Add random noise to current

            # Use the perturbed values in your calculations
            Enernst = E0 + (((R * perturbed_T) / (2 * F)) * (2.303 * (math.log((PH2 * PO2) / (PH2O)))))
            i_limit = ilimit_den / A
            activation_loss = ((R * perturbed_T) / (alpha * n * F)) * (2.303 * (math.log((i_limit / i0_den))))
            concentration_loss = - (((R * perturbed_T) / (n * F)) * (2.303 * (math.log(1 - (i_limit / ilimit_den)))))
            VStack = Enernst - activation_loss - concentration_loss

            X_train.append((perturbed_T, perturbed_current))
            Y_train.append(VStack)

        for current in test_currents:
            # Introduce randomness by perturbing temperature and current
            perturbed_T = T + random.uniform(-1, 1) # Add random noise to temperature
            perturbed_current = current + random.uniform(-10, 10) # Add random noise to current

            # Use the perturbed values in your calculations
            Enernst = E0 + (((R * perturbed_T) / (2 * F)) * (2.303 * (math.log((PH2 * PO2) / (PH2O)))))
            i_limit = ilimit_den / A
            activation_loss = ((R * perturbed_T) / (alpha * n * F)) * (2.303 * (math.log((i_limit / i0_den))))
            concentration_loss = - (((R * perturbed_T) / (n * F)) * (2.303 * (math.log(1 - (i_limit / ilimit_den)))))
            VStack = Enernst - activation_loss - concentration_loss

            X_test.append((perturbed_T, perturbed_current))
            Y_test.append(VStack)

        if len(X_train) >= num_samples:
            break

    random.shuffle(X_train)
    random.shuffle(X_test)
    random.shuffle(Y_train)
    random.shuffle(Y_test)

    return X_train, Y_train, X_test, Y_test

pset = gp.PrimitiveSet("MAIN", arity=2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(math.cos, 1)
pset.addTerminal(1.0)
pset.addEphemeralConstant("rand", lambda: random.uniform(-1, 1))

max_depth = 3

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=max_depth)

random.seed(142)

runs = 50
pop_size = 100
crossover_prob = 0.8
mutation_prob = 0.01
num_generations = 50

mae_train_list, mae_test_list = [], []
mse_train_list, mse_test_list = [] , []

best_individuals, best_equations = [], []

for run in range(runs):
    X_train, Y_train, X_test, Y_test = generate_data()
    toolbox.register("evaluate", evaluate, x_train=X_train, y_train=Y_train, x_test=X_test, y_test=Y_test)

    population = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min_mae_train", np.min)
    stats.register("min_mse_train", np.min)
    stats.register("min_mae_test", np.min)
    stats.register("min_mse_test", np.min)

    algorithms.eaMuPlusLambda(population, toolbox, mu=pop_size, lambda_=2 * pop_size, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations, stats=stats, halloffame=hof, verbose=False)

    best_individual = hof[0]
    best_individuals.append(best_individual)
    mae_train, mse_train, mae_test, mse_test = best_individual.fitness.values

    mae_train_list.append(mae_train)
    mse_train_list.append(mse_train)
    mae_test_list.append(mae_test)
    mse_test_list.append(mse_test)

    best_individual_idx = np.argmin(mae_test_list)
    best_individual_test = best_individuals[best_individual_idx]

    best_equation = str(best_individual_test)
    best_equations.append(best_equation)

all_mae_train, all_mse_train = np.array(mae_train_list), np.array(mse_train_list)
all_mae_test, all_mse_test = np.array(mae_test_list), np.array(mse_test_list)

best_mae_train, worst_mae_train, mean_mae_train, variance_mae_train = calculate_statistics(all_mae_train)
best_mse_train, worst_mse_train, mean_mse_train, variance_mse_train = calculate_statistics(all_mse_train)

best_mae_test, worst_mae_test, mean_mae_test, variance_mae_test = calculate_statistics(all_mae_test)
best_mse_test, worst_mse_test, mean_mse_test, variance_mse_test = calculate_statistics(all_mse_test)

best_individual_test = best_individuals[best_individual_idx]
best_equation = str(best_individual_test)
compiled_function = gp.compile(expr=best_individual_test, pset=pset)

best_result = None
best_T_value = None
best_current_value = None

for T_value, current_value in zip(X_test, Y_test):
    result = compiled_function(T_value, current_value)
    if best_result is None or result < best_result:
        best_result = result
        best_T_value = T_value
        best_current_value = current_value

print("MAE on Training Data \n\n Best: {}\n Worst: {}\n Mean: {}\n Variance: {}\n".format(best_mae_train, worst_mae_train, mean_mae_train, variance_mae_train))
print("MSE on Training Data \n\n Best: {}\n Worst: {}\n Mean: {}\n Variance: {}\n".format(best_mse_train, worst_mse_train, mean_mse_train, variance_mse_train))

print("MAE on Test Data \n\n Best: {}\n Worst: {}\n Mean: {}\n Variance: {}".format(best_mae_test, worst_mae_test, mean_mae_test, variance_mae_test))
print("\nMSE on Test Data \n\n Best: {}\n Worst: {}\n Mean: {}\n Variance: {}".format(best_mse_test, worst_mse_test, mean_mse_test, variance_mse_test))

print("\n Best Value (with respect to test data): {}\n Best Equation (with respect to test data): {}\n".format(best_result, best_equations[np.argmin(mae_test_list)]))


In [ ]:
# Artificial Neural Network

X_train, Y_train, X_test, Y_test = generate_data()
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
train_mae_list, train_mse_list, test_mae_list, test_mse_list = [], [], [], []

model = keras.Sequential([
    keras.layers.Input(shape=(2,)), # Input layer with 2 features (temperature and current)
    keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)), # L2 regularization
    keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)), # L2 regularization
    keras.layers.Dense(1) # Output layer with a single neuron for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_val, Y_val), verbose=0)

train_loss, train_mae = model.evaluate(X_train, Y_train, verbose=0)
train_mae_list.append(train_mae)
train_mse_list.append(train_loss)

test_loss, test_mae = model.evaluate(X_test, Y_test, verbose=0)
test_mae_list.append(test_mae)
test_mse_list.append(test_loss)

Y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(Y_train, Y_train_pred)
train_mse_list.append(train_mse)

Y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(Y_test, Y_test_pred)
test_mse_list.append(test_mse)

weights = model.get_weights()

T = 900
current = 500

input_layer = np.array([T, current])

def get_value(T, current):
    # Use the weights and biases to construct the equation
    hidden_layer1 = np.dot(input_layer, weights[0]) + weights[1]
    hidden_layer1 = np.maximum(0, hidden_layer1) # ReLU activation
    hidden_layer2 = np.dot(hidden_layer1, weights[2]) + weights[3]
    hidden_layer2 = np.maximum(0, hidden_layer2) # ReLU activation
    output = np.dot(hidden_layer2, weights[4]) + weights[5]
    return output

def construct_equation():
    equation = f"output = ReLU({weights[0][0]} * T + {weights[1][0]}) # First hidden layer\n"
    equation += f"output = ReLU({weights[2][0]} * output + {weights[3][0]}) # Second hidden layer\n"
    equation += f"output = {weights[4][0]} * output + {weights[5][0]} # Output layer"
    return equation

print("MAE on Training Data \n\nBest: {}\nWorst: {}\nMean: {} \nVariance: {}\n".format(*calculate_statistics(train_mae_list)))
print("MSE on Training Data \n\nBest: {}\nWorst: {}\nMean: {} \nVariance: {}\n".format(*calculate_statistics(train_mse_list)))
print("MAE on Testing Data \n\nBest: {}\nWorst: {}\nMean: {} \nVariance: {}\n".format(*calculate_statistics(test_mae_list)))
print("MSE on Testing Data \n\nBest: {}\nWorst: {}\nMean: {} \nVariance: {}\n".format(*calculate_statistics(test_mse_list)))

print("\nBest Value (with respect to test data): {}".format(str(get_value(T, current)[0])))

equation = construct_equation()
print("\nBest Equation (with respect to test data): {}".format(construct_equation()))


11/11 [==============================] - 0s 2ms/step
MAE on Training Data 

Best: 0.08477642387151718
Worst: 0.08477642387151718
Mean: 0.08477642387151718 
Variance: 0.0

MSE on Training Data 

Best: 0.010958548096140807
Worst: 0.21881218254566193
Mean: 0.11488536532090136 
Variance: 0.010800783338468789

MAE on Testing Data 

Best: 0.08796335011720657
Worst: 0.08796335011720657
Mean: 0.08796335011720657 
Variance: 0.0

MSE on Testing Data 

Best: 0.011814823093036659
Worst: 0.21966834366321564
Mean: 0.11574158337812615 
Variance: 0.010800771503354453


Best Value (with respect to test data): 1.0639705824631775

Best Equation (with respect to test data): output = ReLU([ 1.23343244e-01 -1.39940143e-01  1.48459494e-01 -4.42455018e-34
 -7.42889866e-02 -4.25265096e-02 -4.34854137e-15  9.92128849e-02
 -2.15619380e-08  2.94536382e-01  1.44217595e-01 -9.29116011e-02
  1.09519735e-01  1.75676770e-34  1.92743793e-01  2.15824246e-01
  2.46098340e-01  4.72209133e-28 -1.20949681e-10  6.21991698e-0